In [1]:
!pip install PyDrive


In [1]:
# Importing necessities

import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf
import cv2
import os
import posenet
import time
import pandas as pd
import numpy as np



In [2]:
# Directory for my files
# Will change this later to read files from drive, so it can run fast on colab

dir_ = "..\..\..\AIML community add\deep learning\\final\mp4"
files = os.listdir(dir_)
file = files[0]
os.path.join(dir_, file)

'..\\..\\..\\AIML community add\\deep learning\\final\\mp4\\bhavesh_ingale_band-stretching.mp4'

In [3]:
# Configuring my cudnn for tensorflow 1
# Needed for my laptop, unsure for others

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [5]:
csv_name = 'my_csv.csv'
with tf.Session() as sess:
    
    # Setting ouput stride to 16
    output_stride = 16
    model_id = 101
    model_cfg, model_outputs = posenet.load_model(model_id, sess)
    
    # change to for loop after testing on single file
    file = files[0]
    action = files[0].split("_")[-1].split(".")[0]
    cap = cv2.VideoCapture(os.path.join(dir_, file))
    input_image, display_image, output_scale = posenet.read_cap(cap)
    
    start = time.time()
    frame_count = 0
    while True:
        
        input_image, display_image, output_scale = posenet.read_cap(cap)

        heatmaps_result, offsets_result, displacement_fwd_result, displacement_bwd_result = sess.run(
            model_outputs,
            feed_dict={'image:0': input_image})

        # print(heatmaps_result.shape) # (1, 33, 58, 17)

        pose_scores, keypoint_scores, keypoint_coords = posenet.decode_multi.decode_multiple_poses(
            heatmaps_result.squeeze(axis=0),
            offsets_result.squeeze(axis=0),
            displacement_fwd_result.squeeze(axis=0),
            displacement_bwd_result.squeeze(axis=0),
            output_stride=output_stride,
            max_pose_detections=10,
            min_pose_score=0.15)

        keypoint_coords *= output_scale

        # TODO this isn't particularly fast, use GL for drawing and display someday...
        overlay_image = posenet.draw_skel_and_kp(
            display_image, pose_scores, keypoint_scores, keypoint_coords,
            min_pose_score=0.15, min_part_score=0.3)
        # setting min_pose_score and min_part_score =0 prints the estimated value

        x = keypoint_coords[0][:,0]
        y = keypoint_coords[0][:,1]
        data = {'action': action, 
                'frame_number': frame_count+1, 
                'input_number': np.arange(1, posenet.NUM_KEYPOINTS+1), 
                'x_inputs': x, 'y_inputs': y}
        temp = pd.DataFrame(data=data)
        
        if csv_name not in os.listdir():
            temp.to_csv(csv_name, 
                        header=True, 
                        index=False)
        else:
            temp.to_csv(csv_name, 
                        mode='a', 
                        header=False, 
                        index=False)
        
        cv2.imshow('posenet', overlay_image)
        frame_count += 1
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # print(cap.get(cv2.CAP_PROP_FPS)) -> frame rate
    print('Average FPS: ', frame_count / (time.time() - start))
    print(frame_count)


OSError: webcam failure or video has ended

In [ ]:
# Later, Wont show video as well

In [10]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

ModuleNotFoundError: No module named 'google.colab'

In [5]:
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [9]:
drive.ListFile({'q': "'root' in parents and trashed=false"})#.GetList()

GoogleDriveFileList({'q': "'root' in parents and trashed=false"})

In [ ]:
drive.